In [142]:
import pandas as pd
import re
from os import listdir

In [149]:
def fix(group):
  if len(group) < 1:
    return group

  if group.values[-1] in ['ขาท', 'ขบาท']:
    group.values[-1] = 'บาท'
  if group.values[-1] == 'บาท':
    group.values[-2] = group.values[-2].replace(',', '').replace('.', '')

  if type(group.values[0]) is not str: print(group.values)
  
  if re.match('^0\d{4}', group.values[0]):
    temp = ''
    for t in group.values[0]:
      if t == '-': return group
      if t != ' ': temp += t
      if len(temp) == 5: temp += '-'
    group.values[0] = temp

  return group


In [151]:
rootDir = './bud-csv/'
for file in sorted(listdir(rootDir)):
    print(rootDir+file)
    if not file.endswith('_raw.tsv'): continue

    df = pd.read_csv(rootDir+file, sep='\t')\
        .sort_values(by=['filename', 'line_num'])\
        .reset_index(drop=True)\
        .dropna()

    df['text'] = df.groupby(['filename', 'line_num']).text.apply(fix)

    df.to_csv(rootDir + file[:file.rfind('_raw')] + '_clean.tsv', sep='\t', index=0)


./bud-csv/65026_clean.tsv
./bud-csv/65026_raw.tsv
./bud-csv/65026_raw_items.csv
./bud-csv/65027_clean.tsv
./bud-csv/65027_raw.tsv
./bud-csv/65028_clean.tsv
./bud-csv/65028_cleaned.tsv
./bud-csv/65028_raw.tsv
./bud-csv/65029_clean.tsv
./bud-csv/65029_raw.tsv
./bud-csv/65030_clean.tsv
./bud-csv/65030_raw.tsv
./bud-csv/65031_clean.tsv
./bud-csv/65031_raw.tsv
./bud-csv/65032_clean.tsv
./bud-csv/65032_raw.tsv
./bud-csv/65033_clean.tsv
./bud-csv/65033_raw.tsv
./bud-csv/65034_clean.tsv
./bud-csv/65034_raw.tsv
./bud-csv/65035_clean.tsv
./bud-csv/65035_raw.tsv
./bud-csv/65036_clean.tsv
./bud-csv/65036_raw.tsv
./bud-csv/65037_clean.tsv
./bud-csv/65037_raw.tsv
./bud-csv/65038_clean.tsv
./bud-csv/65038_raw.tsv
./bud-csv/65039_clean.tsv
./bud-csv/65039_raw.tsv
./bud-csv/65040_raw.tsv
./bud-csv/65041_clean.tsv
./bud-csv/65041_raw.tsv
./bud-csv/65042_raw.tsv
./bud-csv/65043_raw.tsv
./bud-csv/65044_raw.tsv
./bud-csv/65045_raw.tsv
./bud-csv/65046_raw.tsv
./bud-csv/65047_raw.tsv
./bud-csv/65048_raw.tsv
